Capítulo 6

6.4 Patrones de unión.

6.4.3 Unión externa 

6.4.3.2 Unión externa por la derecha

Patrón que permite realizar una unión por la izquierda de dos tablas de datos

In [1]:
%%writefile unionDerecha.py
#!/usr/bin/env python
from mrjob.job import MRJob
import re,sys,os

class unionDerecha(MRJob):
    
    def limpiarNombreArchivo(self,archivo):
        encontradaBarra=False
        tamano=len(archivo)
        posicion=tamano-1
        while encontradaBarra==False or posicion==0:
            if archivo[posicion]=="/":
                encontradaBarra=True
                return archivo[posicion+1:tamano]
            else:
                posicion-=1
        if posicion==0:
            return archivo
        
    def mapper_init(self):
        self.namefile=self.limpiarNombreArchivo(os.getenv('map_input_file')) 
        
    def mapper(self,_,line):
        clave=""
        linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',linea[0])
        if encontrado==None:
            if self.namefile=="tiendas.csv":
                linea.append(self.namefile)
                clave=linea[0]  
                yield clave,linea
            else:
                linea.append(self.namefile) 
                clave=linea[1] 
                yield clave,linea
                
    def reducer(self,key,values):
        listaA=[]
        listaB=[]
        #Llenamos las dos listas
        for valor in values:
            if valor[len(valor)-1]=="tiendas.csv":
                listaA.append(valor)
            else:
                listaB.append(valor)
        
        ##### Union por la derecha #####
        for valorB in listaB:
            if listaA:
                for valorA in listaA:
                    yield valorA, valorB
            else:
                yield "null",valorB
                
if __name__ == '__main__':
    unionDerecha.run()

Writing unionDerecha.py


In [4]:
!python unionDerecha.py archivos_datos/tiendas.csv archivos_datos/articulos_stock.csv

["1", "Santander", "tiendas.csv"]	["1001", "1", "20", "articulos_stock.csv"]
["1", "Santander", "tiendas.csv"]	["1003", "1", "15", "articulos_stock.csv"]
["1", "Santander", "tiendas.csv"]	["1007", "1", "8", "articulos_stock.csv"]
["2", "M\u00e9rida", "tiendas.csv"]	["1002", "2", "10", "articulos_stock.csv"]
["2", "M\u00e9rida", "tiendas.csv"]	["1005", "2", "5", "articulos_stock.csv"]
["2", "M\u00e9rida", "tiendas.csv"]	["1008", "2", "5", "articulos_stock.csv"]
["3", "Granada", "tiendas.csv"]	["1006", "3", "3", "articulos_stock.csv"]
["3", "Granada", "tiendas.csv"]	["1010", "3", "20", "articulos_stock.csv"]
"null"	["1009", "4", "1", "articulos_stock.csv"]
"null"	["1004", "5", "10", "articulos_stock.csv"]


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\unionDerecha.manuel.20201204.193935.181821
Running step 1 of 1...
job output is in C:\Users\cyber\AppData\Local\Temp\unionDerecha.manuel.20201204.193935.181821\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\unionDerecha.manuel.20201204.193935.181821\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\unionDerecha.manuel.20201204.193935.181821...
